# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [18]:
browser = webdriver.Chrome(executable_path='C:\\Users\\user\\AppData\\Local\\Programs\\Python\\Python37-32\\chromedriver')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''
    # Scroll down to the bottom. (模擬網頁下拉)
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height. (檢視目前網頁底端)
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        print("到達頁面底端")
        break

    last_height = new_height
    print("網頁更新中...")

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [19]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [20]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 三星總裁證實「今年沒有Note 21」！原因曝光　最快明年才會推出

記者吳珍儀／綜合報導
三星Galaxy發表會將於8月11日登場，三星總裁盧泰文博士於官網發布最新文章，也證實今年將不會推出Note系列新機。
據外媒提到，三星於今年3月曾提到今年「可能跳過Note系列」，原因是晶片短缺。三星電子執行長高東真指出，Note系列在三星推出的手機中被定位為高階機型，一年中推出兩款旗艦機型可能有些負擔，因此很難在今年發布Note機型。新的Note機型發布的時間可能改成明年。
目前尚不清楚晶片短缺是否也會影響三星今年的其他發布計畫，但三星預計將在今年下半年推出採用旗艦晶片的新折疊手機。
----------------------------------------------------------------------

[1] 台塑生醫又出手！斥資2.47億元參與和康生技私募　申讓聯亞藥1千張

▲台塑生醫斥資2.47億元參與和康生技的私募案。（圖／記者余弦妙攝）
記者吳康瑋／綜合報導
台塑生醫又出手投資了！為能進一步擴大生技領域發展版圖，台塑生醫今（27）日宣佈，正式參與台灣關節注射液及人工代用骨等醫材領導大廠和康生技的私募案，未來將可藉以跨入高階醫材的應用發展，使生技版圖更加完整，也可藉以和甫於7月初取得和康生技經營權的大江生醫策略結盟，一起推動台灣生技大聯盟。
這也是台塑生醫繼投資聯生藥、聯亞藥之後，再次找尋策略性夥伴合作。台塑生醫日前透過2次處分聯亞藥的股份，一共賣出4236張，共收回成本1.79億元，共計進帳未分配盈餘4.88億元。
和康生技是在今日下午重訊公告指出，公司擬洽定台塑生醫以策略性投資人身分，作為本次私募普通股之應募人，這次私募預計發行753萬4235股普通股，應募人應於定價日7月27日起15日內繳足股款共計2.47億元。
台塑生醫董事長王瑞瑜表示，和康生技是台灣最具國際競爭力的醫材廠商之一，掌握各類醫療等級膠原蛋白、玻尿酸的關鍵技術，多達50張的各國上市許可證，搭配大江生醫全球化的發展能力，將可產生巨大的綜和效益，透過此次合作，我們將會一起努力，讓台灣的生技實力再次站上國際的舞台。
據了解，台